# **Load data**

## Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write down which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Code projects\\final-hackathon\\cybersecurity-intrusion-detection\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Code projects\\final-hackathon\\cybersecurity-intrusion-detection'

---

# Initial Setup and Extracting Data

Importing libaries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('data/raw/cybersecurity_intrusion_data.csv')
df

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
0,SID_00001,599,TCP,4,492.983263,DES,0.606818,1,Edge,0,1
1,SID_00002,472,TCP,3,1557.996461,DES,0.301569,0,Firefox,0,0
2,SID_00003,629,TCP,3,75.044262,DES,0.739164,2,Chrome,0,1
3,SID_00004,804,UDP,4,601.248835,DES,0.123267,0,Unknown,0,1
4,SID_00005,453,TCP,5,532.540888,AES,0.054874,1,Firefox,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9532,SID_09533,194,ICMP,3,226.049889,AES,0.517737,3,Chrome,0,1
9533,SID_09534,380,TCP,3,182.848475,NaN,0.408485,0,Chrome,0,0
9534,SID_09535,664,TCP,5,35.170248,AES,0.359200,1,Firefox,0,0
9535,SID_09536,406,TCP,4,86.664703,AES,0.537417,1,Chrome,1,0


## Feature Description
Description of the features were provided with the dataset on [kaggle](https://www.kaggle.com/datasets/dnkumars/cybersecurity-intrusion-detection-dataset). Below is a brief summary of each feature:

| **Feature Name**          | **Category**              | **Description** |
|-----------------------------|----------------------------|-----------------|
| `network_packet_size`       | Network-Based             | Represents the size of network packets (64–1500 bytes). Small packets (~64 bytes) may be control messages; large packets (~1500 bytes) carry bulk data. Abnormally small or large packets can indicate reconnaissance or exploitation. |
| `protocol_type`             | Network-Based             | Communication protocol used in the session: **TCP** (reliable, common for HTTP/HTTPS/SSH), **UDP** (fast, less reliable, used in VoIP/streaming), or **ICMP** (network diagnostics, often abused in DoS attacks). |
| `encryption_used`           | Network-Based             | Encryption protocol: **AES** (strong), **DES** (weak, outdated), or **None** (unencrypted). Attackers might avoid encryption or use weak ones to exploit vulnerabilities. |
| `login_attempts`            | User Behavior-Based       | Number of login attempts. Typical users: 1–3. High values may indicate brute-force attacks with hundreds or thousands of attempts. |
| `session_duration`          | User Behavior-Based       | Session length in seconds. Very long sessions may indicate unauthorized access or an attacker maintaining persistence. |
| `failed_logins`             | User Behavior-Based       | Number of failed login attempts. High values suggest credential stuffing or dictionary attacks. A pattern of many failed attempts followed by a success could indicate compromise. |
| `unusual_time_access`       | User Behavior-Based       | Binary flag (0 or 1) indicating login at an unusual time. Attackers often access systems outside normal business hours to evade detection. |
| `ip_reputation_score`       | User Behavior-Based       | A score (0–1) representing IP trustworthiness. Higher scores indicate more suspicious activity (e.g., botnets, spam, or prior attacks). |
| `browser_type`              | User Behavior-Based       | User’s browser (Chrome, Firefox, Edge, Safari, etc.). Unknown browsers could indicate bots or automated scripts. |
| `attack_detected`           | **Target Variable**       | Binary classification target: **1** = attack detected, **0** = normal activity. |



---

# Data Cleaning
The dataset will be cleaned to handle any missing values, duplicated entries and other inconsistencies.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9537 entries, 0 to 9536
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   session_id           9537 non-null   object 
 1   network_packet_size  9537 non-null   int64  
 2   protocol_type        9537 non-null   object 
 3   login_attempts       9537 non-null   int64  
 4   session_duration     9537 non-null   float64
 5   encryption_used      7571 non-null   object 
 6   ip_reputation_score  9537 non-null   float64
 7   failed_logins        9537 non-null   int64  
 8   browser_type         9537 non-null   object 
 9   unusual_time_access  9537 non-null   int64  
 10  attack_detected      9537 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 819.7+ KB


## Handling Missing Values and Duplicates

In [7]:
df.isna().sum()

session_id                0
network_packet_size       0
protocol_type             0
login_attempts            0
session_duration          0
encryption_used        1966
ip_reputation_score       0
failed_logins             0
browser_type              0
unusual_time_access       0
attack_detected           0
dtype: int64

In [8]:
e_used_unique_counts = df['encryption_used'].value_counts(dropna=False).reset_index()
e_used_unique_counts

,encryption_used,count
0,AES,4706
1,DES,2865
2,NaN,1966


In [9]:
df['encryption_used'] = df['encryption_used'].fillna('No encryption')

In [10]:
df.isna().sum()

session_id             0
network_packet_size    0
protocol_type          0
login_attempts         0
session_duration       0
encryption_used        0
ip_reputation_score    0
failed_logins          0
browser_type           0
unusual_time_access    0
attack_detected        0
dtype: int64

In [11]:
df.duplicated().sum()

0

## Checking for Data Inconsistencies

In [12]:
df.describe()

,network_packet_size,login_attempts,session_duration,ip_reputation_score,failed_logins,unusual_time_access,attack_detected
count,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000,9537.000000
mean,500.430639,4.032086,792.745312,0.331338,1.517773,0.149942,0.447101
std,198.379364,1.963012,786.560144,0.177175,1.033988,0.357034,0.497220
min,64.000000,1.000000,0.500000,0.002497,0.000000,0.000000,0.000000
25%,365.000000,3.000000,231.953006,0.191946,1.000000,0.000000,0.000000
50%,499.000000,4.000000,556.277457,0.314778,1.000000,0.000000,0.000000
75%,635.000000,5.000000,1105.380602,0.453388,2.000000,0.000000,1.000000
max,1285.000000,13.000000,7190.392213,0.924299,5.000000,1.000000,1.000000


In [13]:
df['session_id'].nunique()

9537

In [14]:
for col in ['protocol_type','encryption_used','browser_type']:
    print(df[col].value_counts())
    print("\n")

protocol_type
TCP     6624
UDP     2406
ICMP     507
Name: count, dtype: int64


encryption_used
AES              4706
DES              2865
No encryption    1966
Name: count, dtype: int64


browser_type
Chrome     5137
Firefox    1944
Edge       1469
Unknown     502
Safari      485
Name: count, dtype: int64




In [15]:
for col in ['unusual_time_access', 'attack_detected']:
    print(df[col].value_counts())
    print("\n")

unusual_time_access
0    8107
1    1430
Name: count, dtype: int64


attack_detected
0    5273
1    4264
Name: count, dtype: int64




In [16]:
df[df['login_attempts'] <= df['failed_logins']]

,session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected
7,SID_00008,653,TCP,3,12.599906,DES,0.097719,3,Chrome,1,1
12,SID_00013,548,TCP,2,186.147638,No encryption,0.406899,2,Safari,1,0
14,SID_00015,155,TCP,3,77.849952,No encryption,0.352476,3,Chrome,0,1
17,SID_00018,562,UDP,1,87.641002,No encryption,0.136729,2,Firefox,0,0
21,SID_00022,454,TCP,1,299.711548,No encryption,0.373875,1,Chrome,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9500,SID_09501,446,TCP,1,823.408520,DES,0.661118,3,Edge,0,1
9506,SID_09507,475,UDP,1,25.957496,AES,0.528800,1,Firefox,0,1
9509,SID_09510,665,TCP,3,530.626417,No encryption,0.321948,3,Chrome,0,1
9524,SID_09525,315,UDP,3,1339.802736,DES,0.435285,3,Safari,0,1


In [17]:
df.loc[df["login_attempts"] <= df["failed_logins"], "login_attempts"] = (
    df["login_attempts"] + df["failed_logins"]
)
df[df['login_attempts'] <= df['failed_logins']].shape

(0, 11)

---

# Saving Cleaned Data

In [18]:
df.to_csv('data/processed/cybersecurity_intrusion_data_cleaned.csv', index=False)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.